In [1]:
%load tutorial.db [rw]

In [5]:
Create table If Not Exists Activity (machine_id int, process_id int, activity_type TEXT CHECK( activity_type IN ('start', 'end')), timestamp float)

In [6]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '0', 'start', '0.712')

In [7]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '0', 'end', '1.52')

In [8]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '1', 'start', '3.14')

In [9]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '1', 'end', '4.12')

In [10]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '0', 'start', '0.55')

In [11]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '0', 'end', '1.55')

In [12]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '1', 'start', '0.43')

In [13]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '1', 'end', '1.42')

In [14]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '0', 'start', '4.1')

In [15]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '0', 'end', '4.512')

In [16]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '1', 'start', '2.5')

In [17]:
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '1', 'end', '5')

## There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

In [18]:
select *from activity 

machine_id,process_id,activity_type,timestamp
0,0,start,0.712
0,0,end,1.52
0,1,start,3.14
0,1,end,4.12
1,0,start,0.55
1,0,end,1.55
1,1,start,0.43
1,1,end,1.42
2,0,start,4.1
2,0,end,4.512


expected output

+------------+-----------------+
| machine_id | processing_time |
+------------+-----------------+
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |
+------------+-----------------+

## first approach: groupby and case when

In [21]:
select *
from activity
group by machine_id, process_id

machine_id,process_id,activity_type,timestamp
0,0,start,0.712
0,1,start,3.14
1,0,start,0.55
1,1,start,0.43
2,0,start,4.1
2,1,start,2.5


In [24]:
select machine_id, sum(case when activity_type='start' then timestamp*-1 else timestamp end)/count(distinct process_id) as process_time
from activity
group by machine_id

machine_id,process_time
0,0.894
1,0.995
2,1.456


## second: join (manually with where) and groupby

In [25]:
SELECT *
FROM Activity a, 
     Activity b
WHERE 
    a.machine_id = b.machine_id
AND 
    a.process_id = b.process_id
AND 
    a.activity_type = 'start'
AND 
    b.activity_type = 'end'

machine_id,process_id,activity_type,timestamp,machine_id,process_id,activity_type,timestamp
0,0,start,0.712,0,0,end,1.52
0,1,start,3.14,0,1,end,4.12
1,0,start,0.55,1,0,end,1.55
1,1,start,0.43,1,1,end,1.42
2,0,start,4.1,2,0,end,4.512
2,1,start,2.5,2,1,end,5.0


In [29]:
SELECT a.machine_id, avg(b.timestamp-a.timestamp) as process_time
FROM Activity a, 
     Activity b
WHERE 
    a.machine_id = b.machine_id
AND 
    a.process_id = b.process_id
AND 
    a.activity_type = 'start'
AND 
    b.activity_type = 'end'
group by a.machine_id

machine_id,process_time
0,0.894
1,0.995
2,1.456
